# Flights data cleaning
Erica Landreth

In [0]:
%python
# Restart the Python kernel
dbutils.library.restartPython()

In [0]:
# imports
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import pyspark.sql.functions as F
import pytz
from datetime import datetime, timedelta
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, StructType
from pyspark.sql import types
from pyspark.sql.types import *
from pyspark.sql import Window
import re

## Filtering to relevant rows/columns

In [0]:
display(dbutils.fs.ls(f"dbfs:/mnt/mids-w261/datasets_final_project_2022/"))

In [0]:
display(dbutils.fs.ls(f"dbfs:/mnt/mids-w261/OTPW_12M/OTPW_12M/"))

In [0]:
# load flights data

# dataset = 'parquet_airlines_data_3m' # 3 months
# dataset = 'parquet_airlines_data_1y' # 1 year
# dataset = "OTPW_3M_2015"
dataset = "OTPW_12M_2015"

# df_flights = spark.read.format("csv").option("header","true").load(f"dbfs:/mnt/mids-w261/{dataset}.csv")
df_flights = spark.read.format("csv").option("header","true").load(f"dbfs:/mnt/mids-w261/OTPW_12M/OTPW_12M/{dataset}.csv.gz").cache()
shape_orig = (df_flights.count(), len(df_flights.dtypes))
display(df_flights)
print(f"Original shape: {shape_orig}")

In [0]:
## define columns to drop
# columns related to diversion: not enough data to use the diversion info
div_cols = [col for col in df_flights.columns if col.startswith('DIV') and col != "DIVERTED"]
# redundant carrier ID's (EDA indicated that OP_UNIQUE_CARRIER is sufficient)
xtra_carrier_cols = ["OP_CARRIER_AIRLINE_ID","OP_CARRIER"]
# redundant airport ID's (EDA indicated that ORIGIN/DEST and *_AIRPORT_SEQ_ID are sufficient)
xtra_airport_cols = [ \
  "ORIGIN_AIRPORT_ID","ORIGIN_CITY_MARKET_ID","ORIGIN_STATE_ABR","ORIGIN_STATE_NM","ORIGIN_WAC", \
  "DEST_AIRPORT_ID","DEST_CITY_MARKET_ID","DEST_STATE_ABR","DEST_STATE_NM","DEST_WAC"]
# redundant flight info (could be recreated if need be)
xtra_flight_cols = ["WHEELS_OFF","WHEELS_ON","FLIGHTS","ACTUAL_ELAPSED_TIME","DISTANCE_GROUP"]
# redundant delay status info (could be recreated if need be)
xtra_time_cols = ["DEP_TIME","DEP_DELAY_NEW","DEP_DEL15","DEP_DELAY_GROUP","ARR_TIME","ARR_DELAY_NEW","ARR_DEL15","ARR_DELAY_GROUP"]

## fields to keep
# core features: useful for ML features and/or feature engineering
core_feats = ["FL_DATE","OP_UNIQUE_CARRIER","TAIL_NUM","OP_CARRIER_FL_NUM","ORIGIN","DEST","CRS_DEP_TIME","DEP_DELAY","CRS_ARR_TIME","ARR_DELAY","CANCELLED","DIVERTED","CRS_ELAPSED_TIME","AIR_TIME","DISTANCE"]
# we may or may not end up using these, but they can't easily be recreated later, so we'll keep them to be cautious
on_the_fence = ["ORIGIN_AIRPORT_SEQ_ID","DEST_AIRPORT_SEQ_ID","TAXI_OUT","TAXI_IN"]
# useful for time series analysis
time_series = ["QUARTER","MONTH","DAY_OF_MONTH","DAY_OF_WEEK","DEP_TIME_BLK","ARR_TIME_BLK","YEAR"]
# useful to sanity check that joins are successful
sanity_check = ["ORIGIN_CITY_NAME","DEST_CITY_NAME","ORIGIN_STATE_FIPS","DEST_STATE_FIPS"]
# provides reasoning for cancellations, delays, and returns to gate
delay_info = [col for col in df_flights.columns if col.endswith("_DELAY") and col not in core_feats] + ["CANCELLATION_CODE"] + ["FIRST_DEP_TIME","LONGEST_ADD_GTIME","TOTAL_ADD_GTIME"]
    # Note: cancellation codes are: "A" for carrier-caused, "B" for weather, "C" for National Aviation System, and "D" for security

all_cols = div_cols+xtra_carrier_cols+xtra_airport_cols+xtra_flight_cols+xtra_time_cols+core_feats+on_the_fence+time_series+sanity_check+delay_info

missing = [col for col in df_flights.columns if col not in all_cols]
print(missing)

In [0]:
# define columns to keep
keep_me = core_feats + on_the_fence + time_series + sanity_check + delay_info
keep_me += [c for c in df_flights.columns if "Hourly" in c]
keep_me

In [0]:
keep_me += ['REM', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'STATION', 'Sunrise', 'Sunset'] #shruti wants

In [0]:
df_flights = df_flights.select(keep_me)

In [0]:
# filter to columns of interest, and de-dupe
# df_flights = df_flights.select(keep_me).distinct()

# manual!!! in prev run, saw that the distinct did not do anything
df_flights = df_flights.select(keep_me).cache()

shape_filt = (df_flights.count(), len(df_flights.dtypes))
display(df_flights)
print(f"Filtered shape: {shape_filt}")

In [0]:
# # sanity check: we expect half the records after de-dupe
# shape_orig[0]/shape_filt[0] == 2.0

In [0]:
# # filter to those rows with outcome info populated
# df_flights = df_flights.filter( (col("DEP_DELAY").isNotNull()) | (col("CANCELLED") == 1) | (col("DIVERTED") == 1) ).cache()
# shape_outcome = (df_flights.count(), len(df_flights.dtypes))
# print(f"Shape after filtering for populated outcome vars: {shape_outcome}")

# manual !!! saw that above block did not filter anything in previous run

# filter to those with populated info for basic flight metadata
df_flights = df_flights.filter( (col("OP_UNIQUE_CARRIER").isNotNull()) & (col("ORIGIN").isNotNull()) & (col("DEST").isNotNull()) & (col("FL_DATE").isNotNull()) & (col("CRS_DEP_TIME").isNotNull()) & (col("CRS_ARR_TIME").isNotNull()) ).cache()
shape_basic = (df_flights.count(), len(df_flights.dtypes))
print(f"Shape after filtering for basic feature info: {shape_basic}")

## Converting flight data to UTC time zones

### Create time zone reference file

The below code was used to look up the time zone for each airport. The resulting time zone info was saved out to parquet, so from this point on, just load the time zone parquet (see below for path).

In [0]:
# !pip install timezonefinder

In [0]:
# # imports
# from pyspark.sql.functions import udf
# from pyspark.sql.types import StringType
# from timezonefinder import TimezoneFinder
# import pytz
# from datetime import datetime
# from pyspark.sql.functions import col

# # load stations data
# df_stations = spark.read.parquet(f"dbfs:/mnt/mids-w261/datasets_final_project_2022/stations_data/stations_with_neighbors.parquet/")

# # get unique airport info from stations table
# df_locs = df_stations.select('neighbor_call','neighbor_lat','neighbor_lon').distinct()
# display(df_locs)

# # define function to look up time zones
# def find_timezone(lat, lng):
#     tf = TimezoneFinder()
#     timezone_str = tf.timezone_at(lat=lat, lng=lng)
#     return timezone_str if timezone_str else "Unknown"

# # define udf for time zone lookup
# find_timezone_udf = udf(find_timezone, StringType())

# # add time zone column
# df_locs = df_locs.withColumn("timezone", find_timezone_udf(col("neighbor_lat"), col("neighbor_lon")))

# # save df_time zone info as a parquet file
# folder_path = "dbfs:/student-groups/Group_4_1"
# df_locs.write.parquet(f"{folder_path}/external/tz_lookup.parquet")

In [0]:
# from pyspark.sql import Row

# df_tz = spark.read.parquet(f"dbfs:/student-groups/Group_4_1/external/tz_lookup.parquet")

# # manually fill in missing time zone info
# # note: neighbor_call is ICAO
# BBG = Row(neighbor_call="BBG", neighbor_lat=36.53856729627892, neighbor_lon=-93.19908127077512, timezone="America/Chicago")
# KOGS = Row(neighbor_call="KOGS", neighbor_lat=44.6820707679313, neighbor_lon=-75.47692203483886, timezone="America/New_York")
# NSTU = Row(neighbor_call="NSTU", neighbor_lat=-14.329024376251269, neighbor_lon=-170.71329690482548, timezone="Pacific/Pago_Pago")
# PGSN = Row(neighbor_call="PGSN", neighbor_lat=15.11974288544001, neighbor_lon=145.7282788950688, timezone="Pacific/Saipan")
# PGUM = Row(neighbor_call="PGUM", neighbor_lat=13.48562402083883, neighbor_lon=144.8001485238768, timezone="Pacific/Guam")
# TJPS = Row(neighbor_call="TJPS", neighbor_lat=18.01055087987774, neighbor_lon=-66.56323216254391, timezone="America/Puerto_Rico")
# TJSJ = Row(neighbor_call="TJSJ", neighbor_lat=18.457160454103658, neighbor_lon=-66.0974759565605, timezone="America/Puerto_Rico")
# US_0571 = Row(neighbor_call="US-0571", neighbor_lat=48.25780621107438, neighbor_lon=-103.74169879360201, timezone="America/Chicago")

# man_df = spark.createDataFrame([BBG,KOGS,NSTU,PGSN,PGUM,TJPS,TJSJ,US_0571])
# df_tz = df_tz.union(man_df)


In [0]:
# folder_path = "dbfs:/student-groups/Group_4_1"
# df_tz.write.mode('overwrite').parquet(f"{folder_path}/external/tz_lookup_manually_adjusted.parquet")

### Apply time zones to create full cleaned flights table

In [0]:
df_airports = spark.read.option("header","true").csv(f"dbfs:/mnt/mids-w261/airport-codes_csv.csv")
df_tz = spark.read.parquet(f"dbfs:/student-groups/Group_4_1/external/tz_lookup_manually_adjusted.parquet")

# start with a very, very small flight data sample
tmp_flights = df_flights.limit(10)

# create temporary views
df_flights.createOrReplaceTempView("df_flights")
df_airports.createOrReplaceTempView("df_airports")
df_tz.createOrReplaceTempView("df_tz")

In [0]:
# define all columns of the flights table (for use in SELECT statement)
flights_cols = "x." + ", x.".join(df_flights.columns)

query = f"""

WITH origin AS(
SELECT  {flights_cols},
        x.FL_DATE as date,
        x.CRS_DEP_TIME as dep_time,
        x.CRS_ARR_TIME as arr_time,
        x.ORIGIN as origin_iata,
        x.DEST as dest_iata,
        a.ident as origin_icao
FROM df_flights as x
LEFT JOIN df_airports as a on x.ORIGIN = a.iata_code),

origin_dest AS(
SELECT  {flights_cols},
        x.date,
        x.dep_time,
        x.arr_time,
        x.origin_iata,
        x.dest_iata,
        x.origin_icao,
        a.ident as dest_icao
FROM origin as x
LEFT JOIN df_airports as a on x.dest_iata = a.iata_code),

origin_dest_tz1 AS(
SELECT  {flights_cols},
        x.date,
        x.dep_time,
        x.arr_time,
        x.origin_iata,
        tz.timezone as origin_tz,
        x.dest_iata,
        x.origin_icao,
        x.dest_icao
FROM origin_dest as x
LEFT JOIN df_tz as tz on x.origin_icao = tz.neighbor_call
),

origin_dest_tz2 AS(
SELECT  {flights_cols},
        x.date,
        x.dep_time,
        x.arr_time,
        x.origin_iata,
        x.origin_tz,
        x.dest_iata,
        tz.timezone as dest_tz,
        x.origin_icao,
        x.dest_icao
FROM origin_dest_tz1 as x
LEFT JOIN df_tz as tz on x.dest_icao = tz.neighbor_call
)

SELECT *
FROM origin_dest_tz2

"""

out = spark.sql(query).cache()
display(out)

In [0]:
shape_tz = (out.count(), len(out.dtypes))
print(f"Shape after adding time zone info: {shape_tz}")

In [0]:
# double check that all the time zones were successfully populated
tmp = out.filter( (col("origin_tz").isNull()) | (col("dest_tz").isNull()) )
display(tmp)

In [0]:
def to_utc(yyyymmdd, dep_hhmm, arr_hhmm, dep_tz, arr_tz, flight_dur):
    """
    Create UTC timestamp from flights table columns
    yyyymmdd = FL_DATE
    dep_hhmm = CRS_DEP_TIME
    arr_hhmm = CRS_ARR_TIME
    dep_tz = time zone from time zone table
    arr_tz = time zone from time zone table
    flight_dur = CRS_ELAPSED_TIME (for sanity check of arrival time)

    Returns UTC time stamp, (cast to string)
    """

    dep_hhmm = int(dep_hhmm)
    arr_hhmm = int(arr_hhmm)

    yyyy,MM,dd = yyyymmdd.split('-')
    yyyy = int(yyyy) # get year
    MM = int(MM) # get month
    dd = int(dd) # get day

    dep_hh = dep_hhmm//100 # get hour
    dep_mm = dep_hhmm%100 # get minute
    if dep_hh == 24:
        dep_hh = 0
        dep_shift = True
    else:
        dep_shift = False

    arr_hh = arr_hhmm//100 # get hour
    arr_mm = arr_hhmm%100
    if arr_hh == 24:
        arr_hh = 0
        arr_shift = True
    else:
        arr_shift = False

    # create datetime variable for departure
    dt_dep = datetime(yyyy,MM,dd,dep_hh,dep_mm)
    if dep_shift:
        dt_dep += timedelta(days=1)
    # apply local time zone
    dep_local = pytz.timezone(dep_tz).localize(dt_dep)
    # convert to UTC
    dep_utc = dep_local.astimezone(pytz.utc)

    # create datetime variable for arrival
    dt_arr = datetime(yyyy,MM,dd,arr_hh,arr_mm)
    if arr_shift:
        dt_arr += timedelta(days=1)
    # apply local time zone
    arr_local = pytz.timezone(arr_tz).localize(dt_arr)
    # convert to UTC
    arr_utc = arr_local.astimezone(pytz.utc)

    if dep_utc > arr_utc:
        arr_utc += timedelta(days=1)

    # # sanity check
    # arr_utc_SC = dep_utc + timedelta(minutes=flight_dur)

    dt_format = "%Y-%m-%dT%H:%M:%S"

    # return UTC datetime, cast to string
    # return (dep_utc.strftime(dt_format), arr_utc.strftime(dt_format), arr_utc_SC.strftime(dt_format))
    return (dep_utc.strftime(dt_format), arr_utc.strftime(dt_format))

schema = StructType([
    StructField("dep_datetime", StringType(), False),
    StructField("arr_datetime", StringType(), False),
    # StructField("arr_datetime_SANITYCHECK", StringType(), False)
])

dt_udf = udf(to_utc, schema)

out = out.withColumn('processed', dt_udf(col("date"), col("dep_time"), col("arr_time"), col("origin_tz"), col("dest_tz"), col("CRS_ELAPSED_TIME"))).cache()

cols = [c for c in out.columns if c != "processed"]
# cols += ["processed.dep_datetime","processed.arr_datetime","processed.arr_datetime_SANITYCHECK"]
cols += ["processed.dep_datetime","processed.arr_datetime"]
out = out.select(cols).cache()

# out = out.withColumn("dep_timestamp", F.to_timestamp(col('dep_datetime')).alias('dep_timestamp'))
# out = out.withColumn("arr_timestamp", F.to_timestamp(col('arr_datetime')).alias('arr_timestamp'))
# out = out.withColumn("arr_timestamp_SANITYCHECK", F.to_timestamp(col('arr_datetime_SANITYCHECK')).alias('arr_timestamp_SANITYCHECK'))

display(out)

In [0]:
# remove redundant variables that were added during the join process
redundant = ["date","dep_time","arr_time"]
# note ORIGIN and DEST are technically redudant, but will keep for now

out = out.drop(*redundant).cache()

In [0]:
# final shape
shape_final = (out.count(), len(out.dtypes))
print(f"Shape after cleaning: {shape_final}")

display(out)

## Write cleaned output to parquet

In [0]:
dataset = "OTPW_12M_2015"
folder_path = "dbfs:/student-groups/Group_4_1"
out.write.mode("overwrite").parquet(f"{folder_path}/interim/{dataset}_clean.parquet")

In [0]:
# check that write was successful
df = spark.read.parquet(f"{folder_path}/interim/{dataset}_clean.parquet")
display(df)

In [0]:
display(dbutils.fs.ls(f"{folder_path}/interim/"))

# Weather Data Cleaning

## Double checks nulls/duplicates

In [0]:
dataset = "OTPW_12M_2015"
df_flights = spark.read.format("csv").option("header","true").load(f"dbfs:/mnt/mids-w261/OTPW_12M/OTPW_12M/{dataset}.csv.gz")
display(df_flights.limit(10))

In [0]:
display(out)

In [0]:
display(out
        .filter(F.col('CANCELLED')==0)
        .groupBy('TAIL_NUM','dep_datetime')
        .count()
        .filter(F.col('count') > 1))

In [0]:
display(out
        .filter(F.col('CANCELLED')==0)
        .filter((F.col('DEP_DELAY') == 0) | F.col('DEP_DELAY').isNull())
        .groupBy('TAIL_NUM','dep_datetime')
        .count()
        .filter(F.col('count') > 1))

N812SK	2015-04-07T17:15:00	2

In [0]:

display(out.filter(F.col('TAIL_NUM')=='N812SK').filter(F.col('FL_DATE')=='2015-04-07'))

In [0]:
display(out.filter(F.col('TAIL_NUM')=='N928WN').filter(F.col('FL_DATE')=='2015-03-28'))

Duplicates based on tail number/datetime always occur due to cancellations or due to delays which cause replicated looking datetimes.

## Interpolation

### METAR (Whole df)

In [0]:
dataset = "OTPW_12M_2015"
folder_path = "dbfs:/student-groups/Group_4_1"
df = spark.read.parquet(f"{folder_path}/interim/{dataset}_clean.parquet")
display(df)

Pull wind data from REM

In [0]:


df_interpolate = (
    df
    .withColumn(
        "HourlyWindSpeed",
        F.when(
            F.col("HourlyWindSpeed").isNull(),
            # Extract sustained wind speed from METAR groups
            F.regexp_extract(
                F.col("REM"),
                r'\b(\d{3})(\d{2,3})(?:G(\d{2,3}))?KT\b',  # Regex pattern
                2  # Capture group for sustained wind speed
            ).cast("int")
        ).otherwise(F.col("HourlyWindSpeed"))
    )
    .withColumn(
        "HourlyWindGustSpeed",
        F.when(
            F.col("HourlyWindGustSpeed").isNull(),
            F.greatest(
                # Regular wind gust (G group)
                F.regexp_extract(
                    F.col("REM"),
                    r'\b(\d{3})(\d{2,3})(?:G(\d{2,3}))?KT\b',
                    3
                ).cast("int"),
                # Peak wind gust (PK WND group)
                F.regexp_extract(
                    F.col("REM"),
                    r'PK WND (\d{3})(\d{2,3})/(\d{4})',  # PK WND pattern
                    2  # Capture group for peak wind speed
                ).cast("int")
            )
        ).otherwise(F.col("HourlyWindGustSpeed"))
    )
)




In [0]:
df_interpolate = (df_interpolate \
    .withColumn(
        'HourlyPrecipitation',
        F.when(
            (F.col("HourlyPrecipitation").isNull()) | (F.col("HourlyPrecipitation") == '*'),
            (F.regexp_extract(F.col("REM"), r" P(\d+)", 1).cast("int") * 0.01) # hundredths of inch kept in "remarks" section
        ).otherwise(F.col("HourlyPrecipitation"))
    ) \
    .withColumn('HourlyPrecipitation', F.regexp_replace('HourlyPrecipitation', 'T', '0.01')) \
    .withColumn(
        'HourlyPrecipitation',
        F.regexp_extract('HourlyPrecipitation', r"[0-9]+(\.[0-9]+)?", 0) # Match digits
    ) \
    .withColumn('HourlyPrecipitation', F.col('HourlyPrecipitation').cast(DoubleType())))

We can see that this did not significantly improve null counts

In [0]:

print(f"Original hourly wind speed null count: {df.filter(F.col('HourlyWindSpeed').isNull()).count()}")
print(f"After REM extraction: {df_interpolate.filter(F.col('HourlyWindSpeed').isNull()).count()}")

In [0]:
print(f"Original hourly wind gust speed null count: {df.filter(F.col('HourlyWindGustSpeed').isNull()).count()}")
print(f"After REM extraction: {df_interpolate.filter(F.col('HourlyWindGustSpeed').isNull()).count()}")

In [0]:
print(f"Original precipitation null count: {df.filter(F.col('HourlyPrecipitation').isNull()).count()}")
print(f"After REM extraction: {df_interpolate.filter(F.col('HourlyPrecipitation').isNull()).count()}")

### Exponential smoothing window (fold-based)

In [0]:
df_interpolate = df_interpolate.repartition("STATION", F.date_trunc("hour", "dep_datetime"))


In [0]:

def exponential_smoothing_udf(alpha: float):
    """exponential smoother"""
    def _smooth(values):
        if not values:
            return 0.0  # Default to 0.0 if no non-null values
        total = 0.0
        sum_weights = 0.0
        for idx, value in enumerate(reversed(values)):
            if value is not None:
                weight = (1 - alpha) ** idx
                total += value * weight
                sum_weights += weight
        return total / sum_weights if sum_weights != 0 else 0.0
    return F.udf(_smooth, DoubleType())


In [0]:
#go back to apply -2 hour constraint for test set(s) (?)
def smooth_column(df, col_name, alpha=0.5, window_size=6):
    """Apply exponential smoothing to a single column;
    if there are no non-null col values, return 0."""
    window_spec = Window.partitionBy("STATION") \
                       .orderBy("dep_datetime") \
                       .rowsBetween(-window_size, -1)
    
    return df.withColumn(
        f"non_null_{col_name}",
        F.expr(f"""
            slice(
                filter(
                    collect_list({col_name}) OVER (PARTITION BY STATION ORDER BY dep_datetime),
                    x -> x is not null
                ),
                greatest(
                    size(
                        filter(
                            collect_list({col_name}) OVER (PARTITION BY STATION ORDER BY dep_datetime),
                            x -> x is not null
                        )
                    ) - {window_size - 1},
                    1
                ),
                {window_size}
            )
        """)
    ).withColumn(
        f"smoothed_{col_name}",
        exponential_smoothing_udf(alpha)(F.col(f"non_null_{col_name}"))
    ).withColumn(
        col_name,
        F.coalesce(F.col(col_name), F.col(f"smoothed_{col_name}"), lit(0.0))
    ).drop(f"non_null_{col_name}", f"smoothed_{col_name}")


In [0]:
hourly_cols = [col for col in df_interpolate.columns if col.startswith("Hourly")] #assumed cols that need interpolation

def smooth_multiple_columns(df, columns=[col for col in df.columns if col.startswith("Hourly")], alpha=0.5, window_size=6):
    """Apply smoothing to multiple columns; params:
            alpha: exponential decay factor
            window_size: number of non-null previous records to use"""
    for col in columns:
        df = smooth_column(df, col, alpha, window_size)
    return df


#### sandbox

In [0]:


ALPHA = 0.5  # Smoothing factor
NON_NULL_WINDOW_SIZE = 6  # Use last 6 non-null values

# Window to collect all historical non-null values up to current row
window_collect = Window.partitionBy("STATION") \
                      .orderBy("dep_datetime") \
                      .rowsBetween(-6, -1)

# Step 1: Collect non-null values and extract last 6
df_with_non_null = df_interpolate.withColumn(
    "non_null_precip",
    F.expr(f"""
        slice(
            filter(
                collect_list(HourlyPrecipitation) OVER (PARTITION BY STATION ORDER BY dep_datetime),
                x -> x is not null
            ),
            greatest(
                size(
                    filter(
                        collect_list(HourlyPrecipitation) OVER (PARTITION BY STATION ORDER BY dep_datetime),
                        x -> x is not null
                    )
                ) - {NON_NULL_WINDOW_SIZE - 1},
                1
            ),
            {NON_NULL_WINDOW_SIZE}
        )
    """)
)

# Step 2: Apply exponential smoothing UDF
def exponential_weighted_avg(precip_list):
    total = 0.0
    sum_weights = 0.0
    for idx, value in enumerate(reversed(precip_list)):
        weight = (1 - ALPHA) ** idx  # idx=0 (most recent), idx=5 (oldest of 6)
        total += value * weight
        sum_weights += weight
    return total / sum_weights if sum_weights != 0 else None

exponential_udf = F.udf(exponential_weighted_avg, DoubleType())

df_smoothed = df_with_non_null.withColumn(
    "smoothed_precip",
    exponential_udf("non_null_precip")
).withColumn(
    "HourlyPrecipitation",
    F.coalesce(F.col("HourlyPrecipitation"), F.col("smoothed_precip"),lit(0.0))
).drop("non_null_precip", "smoothed_precip")

display(df_smoothed)


In [0]:
display(df_interpolate.filter(F.col("HourlyPrecipitation").isNull()).limit(10))

In [0]:
df_interpolate.filter(F.col('STATION')=='72645704825').filter(F.col('HourlyPrecipitation').isNotNull()).count()

In [0]:
display(df_smoothed.filter(F.col('STATION')=='72645704825').filter(F.col('dep_datetime')=='2015-07-22T22:27:00'))

### NN-based

In [0]:
stations = spark.read.parquet(f"dbfs:/mnt/mids-w261/datasets_final_project_2022/stations_data/*")


In [0]:
nn_interpolate = df_interpolate.repartition(1000, "STATION")
stations = stations.repartition(1000, "station_id")


Toy example: Precipitation

In [0]:
missing = nn_interpolate.filter(F.col("HourlyPrecipitation").isNull()) \
                   .select("STATION", "dep_datetime", "HourlyPrecipitation")
non_missing = nn_interpolate.filter(F.col("HourlyPrecipitation").isNotNull())

In [0]:
display(missing_neighbors.limit(10))

In [0]:

nn_interpolate = nn_interpolate.repartition("STATION", F.date_trunc("hour", "dep_datetime"))
stations = stations.repartition("station_id")



#filter and cache core datasets
non_missing = nn_interpolate.filter(F.col("HourlyPrecipitation").isNotNull()) \
                           .select("STATION", "dep_datetime", "HourlyPrecipitation") \
                           .persist(StorageLevel.DISK_ONLY)

missing = nn_interpolate.filter(F.col("HourlyPrecipitation").isNull()) \
                        .select("STATION", "dep_datetime", "HourlyPrecipitation")

#time bucketing
time_window = 4 * 3600  # 4 hours in seconds
non_missing_time = non_missing.withColumn(
    "time_bucket",
    F.expr("cast((cast(dep_datetime as long) / 7200) as int) * 7200")  # 2hr buckets
).repartition("STATION", "time_bucket")


In [0]:
# join missing stations with neighbor IDs
missing_neighbors = missing.alias("m").join(
    stations.alias("s"),
    F.col("m.STATION") == F.col("s.station_id"),
    "left"
).select(
    "m.STATION",
    "m.dep_datetime",
    "s.neighbor_id",
    "m.HourlyPrecipitation"
)


In [0]:
non_missing_time=non_missing_time.repartition("STATION", "time_bucket")


missing_neighbors = missing_neighbors.repartition("neighbor_id")


In [0]:



# get non-missing neighbors 
imputed = (
    missing_neighbors.alias("mn")
    .join(
        non_missing_time.alias("nn"),
        (F.col("mn.neighbor_id") == F.col("nn.STATION")) &  # join on neighbor ID
        (F.col("mn.dep_datetime").cast("long") - F.col("nn.dep_datetime").cast("long")).between(7200, 14400),
        "left"
    )
    .withColumn("time_diff", F.abs(F.col("mn.dep_datetime") - F.col("nn.dep_datetime")))
    .withColumn("rnk", F.row_number().over(
        Window.partitionBy("mn.STATION", "mn.dep_datetime")
              .orderBy(F.asc("time_diff"))
    ))
    .filter(F.col("rnk") == 1)
    .select(
        "mn.STATION",
        "mn.dep_datetime",
        F.coalesce("mn.HourlyPrecipitation", "nn.HourlyPrecipitation").alias("HourlyPrecipitation")
    )
)


In [0]:
display(non_missing_time.limit(10))

In [0]:
# 2. Align partitioning for spatial-temporal joins
nn_interpolate = nn_interpolate.repartition(1000, "STATION", F.date_trunc("hour", "dep_datetime"))
stations = stations.repartition(1000, "station_id")

# 3. Filter datasets without caching
non_missing = nn_interpolate.filter(F.col("HourlyPrecipitation").isNotNull()) \
                           .select("STATION", "dep_datetime", "HourlyPrecipitation")

missing = nn_interpolate.filter(F.col("HourlyPrecipitation").isNull()) \
                        .select("STATION", "dep_datetime", "HourlyPrecipitation")

# 4. Time bucketing (for temporal alignment)
time_window = 4 * 3600  # 4 hours in seconds
non_missing_time = non_missing.withColumn(
    "time_bucket",
    F.expr("cast((cast(dep_datetime as long) / 7200) as int) * 7200")  # 2-hour buckets
).repartition(1000, "STATION", "time_bucket")

# 5. Join missing stations with neighbor IDs
missing_neighbors = missing.alias("m").join(
    stations.alias("s"),
    F.col("m.STATION") == F.col("s.station_id"),
    "left"
).select(
    "m.STATION",
    "m.dep_datetime",
    "s.neighbor_id",
    "m.HourlyPrecipitation"
)

# 6. Temporal join with range constraints
imputed = (
    missing_neighbors.alias("mn")
    .join(
        non_missing_time.alias("nn"),
        (F.col("mn.neighbor_id") == F.col("nn.STATION")) & 
        (F.col("mn.dep_datetime").cast("long") - F.col("nn.dep_datetime").cast("long")).between(7200, 14400),
        "left"
    )
    .withColumn("time_diff", F.abs(F.col("mn.dep_datetime").cast("long") - F.col("nn.dep_datetime").cast("long")))
    .withColumn(
        "rnk",
        F.row_number().over(
            Window.partitionBy("mn.STATION", "mn.dep_datetime")
                  .orderBy(F.asc("time_diff"))
        )
    )
    .filter(F.col("rnk") == 1)
    .select(
        "mn.STATION",
        "mn.dep_datetime",
        F.coalesce(F.col("mn.HourlyPrecipitation"), F.col("nn.HourlyPrecipitation")).alias("HourlyPrecipitation")
    )
)


In [0]:
display(imputed.limit(10))

In [0]:
display(imputed.filter(F.col('HourlyPrecipitation').isNotNull()))

In [0]:
display(imputed)

In [0]:
nn_interpolate = nn_interpolate.withColumn(
    "dep_datetime", 
    F.to_timestamp(F.col("dep_datetime"))  # Convert to timestamp if stored as string
)

In [0]:
display(nn_interpolate.limit(10))

In [0]:
#naive (non-weighted) forward-smooth window spec
# window_spec = Window.partitionBy("STATION") \
#                    .orderBy("dep_datetime") \
#                    .rowsBetween(-8, -1)

# df_smoothed = df_interpolate.withColumn(
#     "HourlyPrecipitation",
#     F.when(F.col('HourlyPrecipitation').isNull(),
#     F.coalesce(
#         F.col("HourlyPrecipitation"),
#         F.avg(F.col("HourlyPrecipitation")).over(window_spec)
#     )
# ).otherwise(F.col('HourlyPrecipitation'))
# )

In [0]:

# Define exponential smoothing parameters
ALPHA = 0.3  # Smoothing factor (0 < ALPHA < 1) - need to tune
WINDOW_SIZE = 12

# Create window to collect past [window_size] values (excluding current row)
window_spec = Window.partitionBy("STATION") \
                   .orderBy("dep_datetime") \
                   .rowsBetween(-WINDOW_SIZE, -1)

# Step 1: Collect past precipitation values into a list
df_with_list = df_interpolate.withColumn(
    "past_precip",
    F.collect_list("HourlyPrecipitation").over(window_spec)
)

# UDF for exponential smoothing
def exponential_weighted_avg(precip_list):
    if not precip_list:
        return None
    total = 0.0
    sum_weights = 0.0
    # Reverse to process newest first (higher weights)
    for idx, value in enumerate(reversed(precip_list)):
        if value is not None:
            weight = (1 - ALPHA) ** idx  # idx=0 (newest), idx=6 (oldest)
            total += value * weight
            sum_weights += weight
    return total / sum_weights if sum_weights != 0 else None

exponential_udf = F.udf(exponential_weighted_avg, DoubleType())

# Apply smoothing and fill nulls
df_smoothed = df_with_list.withColumn(
    "smoothed_precip",
    exponential_udf("past_precip")
).withColumn(
    "HourlyPrecipitation",
    F.coalesce(F.col("HourlyPrecipitation"), F.col("smoothed_precip"))
).drop("past_precip", "smoothed_precip")

# Show results
display(df_smoothed)

In [0]:
df_smoothed.filter(F.col('HourlyPrecipitation').isNull()).count()

In [0]:
nn = df_smoothed \
    .join(stations, F.col('STATION') == stations.station_id, "left")


In [0]:
fill = nn.filter(F.col('HourlyPrecipitation').isNotNull()) \
    .select(F.col('STATION').alias('fill_station'), F.col('HourlyPrecipitation').alias('fill_HourlyPrecipitation'))

empty = nn.filter(F.col('HourlyPrecipitation').isNull()).alias('df1') \
    .join(fill.alias('fill'), F.col('df1.neighbor_id') == F.col('fill.fill_station'), 'left') \
    .withColumn('HourlyPrecipitation', F.coalesce(F.col('df1.HourlyPrecipitation'), F.col('fill.fill_HourlyPrecipitation')))



In [0]:
empty.filter(F.col('HourlyPrecipitation').isNull()).count()

In [0]:
display(fill)

### Time-based 